<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled169.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install qiskit-aer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import pickle
from datetime import datetime
from google.colab import output
from tqdm.notebook import tqdm
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from google.colab import drive
drive.mount('/content/drive')

# --- UNCHANGED CLASSES (QuantumRandomGenerator, Environment, Protoribosome, Colony) ---
# NOTE: The Colony.step() method includes the previous fix to advance the environment's time.
class QuantumRandomGenerator:
    def __init__(self, num_bits_precision=64):
        self.simulator = AerSimulator()
        self.num_bits = num_bits_precision
    def get_seed(self):
        qc = QuantumCircuit(self.num_bits, self.num_bits)
        qc.h(range(self.num_bits))
        qc.measure(range(self.num_bits), range(self.num_bits))
        job = self.simulator.run(transpile(qc, self.simulator), shots=1)
        bit_string = list(job.result().get_counts(0).keys())[0]
        return int(bit_string, 2)

class Environment:
    def __init__(self, rng, days_per_cycle=50):
        self.days_per_cycle, self.time, self.rng = days_per_cycle, 0, rng
        self.steps_per_year = 365 * self.days_per_cycle
        self.uv_intensity, self.temperature, self.yearly_uv_severity = self._generate_cycles(num_years=50)

    def _generate_cycles(self, num_years):
        daily_uv_cycle = np.sin(np.linspace(0, 2 * np.pi, self.days_per_cycle)); daily_uv_cycle[daily_uv_cycle < 0] = 0
        daily_temp_swing = -4 * np.cos(np.linspace(0, 2 * np.pi, self.days_per_cycle))
        full_uv, full_temp, yearly_severity_log = [], [], []
        total_steps = num_years * self.steps_per_year
        base_weather_noise = self.rng.standard_normal(total_steps)
        weather_window_size = 14 * self.days_per_cycle
        smoothing_window = np.ones(weather_window_size) / weather_window_size
        weather_pattern = np.convolve(base_weather_noise, smoothing_window, 'same') * 5.0
        global_step_counter = 0
        for _ in range(num_years):
            uv_severity = self.rng.uniform(0.6, 1.4)
            yearly_severity_log.extend([uv_severity] * self.steps_per_year)
            seasonal_cycle = np.sin(np.linspace(0, 2 * np.pi, self.steps_per_year))
            seasonal_temp_base = 15 - 10 * np.cos(np.linspace(0, 2 * np.pi, self.steps_per_year))
            seasonal_amplitude_mod = (0.225 * seasonal_cycle + 0.725) * uv_severity
            GEOTHERMAL_BASE_TEMP = 4.0
            daily_temp_anomaly = 0
            for i in range(self.steps_per_year):
                if i % self.days_per_cycle == 0: daily_temp_anomaly = self.rng.uniform(-1.5, 1.5)
                cloud_cover_factor = self.rng.uniform(0.7, 1.0)
                daily_uv = daily_uv_cycle[i % self.days_per_cycle] * seasonal_amplitude_mod[i] * cloud_cover_factor
                full_uv.append(daily_uv)
                solar_temp = seasonal_temp_base[i] + daily_temp_swing[i % self.days_per_cycle] + weather_pattern[global_step_counter] + daily_temp_anomaly
                full_temp.append(max(GEOTHERMAL_BASE_TEMP, solar_temp))
                global_step_counter += 1
        return np.array(full_uv), np.array(full_temp), np.array(yearly_severity_log)

    def get_current_uv(self): return self.uv_intensity[self.time % len(self.uv_intensity)]
    def get_current_temperature(self): return self.temperature[self.time % len(self.temperature)]
    def step(self): self.time += 1

class Protoribosome:
    def __init__(self, env, strategy, initial_dna, rng, initial_mass=100.0):
        self.env, self.strategy, self.rng = env, strategy, rng
        self.dna_sequence = list(initial_dna)
        self.rna_sequence = list(initial_dna)
        self.rna_damage_level, self.location, self.status = 0.0, 'shadow_zone', 'ACTIVE'
        self.rna_mass, self.atp_pool = initial_mass, 500.0
        self.uv_protection_pool = 50.0
        self.trna_pool = {'Trp': 0, 'Tyr': 0, 'Cys': 0, 'Phe': 0, 'Met': 0, 'His': 0, 'Stop': 0}
        self.dna_mutation_rate, self.gene_conversion_prob = 0.0001, 0.0005
        self.MIN_RNA_LENGTH = 12

    def _transcribe_and_synthesize_trna(self):
        dna_str = "".join(self.dna_sequence)
        trna_genes = {
            'GGT': 'Trp', 'TAT': 'Tyr', 'TGT': 'Cys', 'TTT': 'Phe', 'ATG': 'Met', 'CAT': 'His', 'TTA': 'Stop'
        }
        for gene, trna in trna_genes.items():
            if gene in dna_str: self.trna_pool[trna] += 2
        self.rna_sequence = list(dna_str.replace('T', 'U'))

    def calculate_protection_score(self):
        protection_weights = {
            'UGG': 5.0, 'UAU': 3.0, 'UAC': 3.0, 'UUU': 1.5, 'UUC': 1.5,
            'UGU': 1.0, 'UGC': 1.0, 'AUG': 0.75, 'CAU': 0.5, 'CAC': 0.5,
        }
        baseline_score_per_amino = 0.1
        num_codons = len(self.rna_sequence) // 3
        score = num_codons * baseline_score_per_amino
        score += sum(protection_weights.get("".join(self.rna_sequence[i:i+3]), 0) for i in range(0, len(self.rna_sequence), 3))
        return score

    def has_stop_codon(self):
        return any("".join(self.rna_sequence[i:i+3]) in {'UAA', 'UAG', 'UGA'} for i in range(0, len(self.rna_sequence), 3))

    def _synthesize_protein(self):
        if self.status != 'ACTIVE': return
        has_stop = self.has_stop_codon()
        if self.strategy == 'cautious' and has_stop and self.trna_pool['Stop'] > 0:
            self.status = 'ARRESTED'; self.trna_pool['Stop'] -= 1; return
        if self.strategy == 'readthrough' and has_stop:
            self.status = 'INACTIVE'; return

        trna_map = {'UGG':'Trp','UAU':'Tyr','UAC':'Tyr','UGU':'Cys','UGC':'Cys','UUU':'Phe','UUC':'Phe','AUG':'Met','CAU':'His','CAC':'His'}
        required_trnas = {}
        for i in range(0, len(self.rna_sequence), 3):
            codon = "".join(self.rna_sequence[i:i+3])
            if (trna_type := trna_map.get(codon)):
                 required_trnas[trna_type] = required_trnas.get(trna_type, 0) + 1

        if all(self.trna_pool.get(t, 0) >= count for t, count in required_trnas.items()):
            for t, count in required_trnas.items(): self.trna_pool[t] -= count
            score = self.calculate_protection_score()
            self.uv_protection_pool += 0.20 * score
            self.atp_pool -= 0.02 * len(self.rna_sequence)

    def _replicate_dna(self):
        if self.status != 'ACTIVE' or self.rna_mass < 80 or self.atp_pool < 20: return None
        offspring_mass = self.rna_mass * 0.5; self.rna_mass -= offspring_mass
        self.atp_pool -= 20

        new_dna = list(self.dna_sequence)
        for i in range(len(new_dna)):
            base = new_dna[i]
            effective_mutation_rate = self.dna_mutation_rate * 1.5 if base in ('A', 'T') else self.dna_mutation_rate * 0.5
            if self.rng.random() < effective_mutation_rate:
                new_dna[i] = self.rng.choice(list("ATGC"))

        if self.rng.random() < self.dna_mutation_rate / 2:
            new_dna.insert(self.rng.integers(0, len(new_dna) + 1), self.rng.choice(list("ATGC")))
        if len(new_dna) > self.MIN_RNA_LENGTH and self.rng.random() < self.dna_mutation_rate / 2:
            new_dna.pop(self.rng.integers(0, len(new_dna)))

        trim_len = len(new_dna) - (len(new_dna) % 3)
        return Protoribosome(self.env, self.strategy, "".join(new_dna[:trim_len]), self.rng, offspring_mass)

    def _perform_gene_conversion(self):
        if self.atp_pool > 200 and self.uv_protection_pool > 100 and self.rng.random() < self.gene_conversion_prob:
            self.atp_pool -= 150
            self.dna_sequence = list("".join(self.rna_sequence).replace('U', 'T'))

    def step(self, current_uv):
        if self.status == 'INACTIVE': return None
        self._transcribe_and_synthesize_trna()
        if self.status == 'ARRESTED' and not self.has_stop_codon(): self.status = 'ACTIVE'

        protection = self.uv_protection_pool * 0.01
        effective_uv = current_uv * max(0.01, 1 - protection)
        self.rna_damage_level += effective_uv * (1.2 if self.strategy == 'cautious' else 1.6)
        self.rna_damage_level = max(0, self.rna_damage_level - 0.5)

        if self.atp_pool < 0.1 or self.rna_damage_level > 3.0 or len(self.rna_sequence) < self.MIN_RNA_LENGTH:
            self.status = 'INACTIVE'; return None

        self._synthesize_protein()

        self.atp_pool = (self.atp_pool + 1.5) * (1 - 0.025)
        if self.location == 'sunlight_zone' and current_uv > 0:
            self.atp_pool += 0.15 * self.uv_protection_pool * current_uv

        self._perform_gene_conversion()
        return self._replicate_dna()

class Colony:
    def __init__(self, env, initial_dna, initial_pop_size, steps_per_day, rng, shuffle_rng):
        self.env, self.initial_dna, self.steps_per_day = env, initial_dna, steps_per_day
        self.max_population, self.rng, self.shuffle_rng = 5000, rng, shuffle_rng
        self.active_population = [Protoribosome(env, 'cautious' if i%2==0 else 'readthrough', initial_dna, rng) for i in range(initial_pop_size)]
        self.inactive_population = []

    def step(self, current_step):
        current_uv = self.env.get_current_uv()
        if current_step > 0 and current_step % self.steps_per_day == 0: self.inactive_population.clear()

        next_gen_and_survivors = []
        for p in self.active_population:
            offspring = p.step(current_uv)
            if p.status != 'INACTIVE':
                next_gen_and_survivors.append(p)
            else:
                self.inactive_population.append(p)
            if offspring:
                next_gen_and_survivors.append(offspring)

        self.active_population = next_gen_and_survivors
        if len(self.active_population) > self.max_population:
            self.shuffle_rng.shuffle(self.active_population)
            self.active_population = self.active_population[:self.max_population]

        self.env.step()

    def get_aggregated_data(self):
        data = {
            'cautious': 0, 'readthrough': 0, 'avg_dna_len': 0.0,
            'avg_tRNA_Trp': 0.0, 'avg_tRNA_Tyr': 0.0, 'avg_tRNA_Cys': 0.0,
            'avg_tRNA_Phe': 0.0, 'avg_tRNA_Met': 0.0, 'avg_tRNA_His': 0.0, 'avg_tRNA_Stop': 0.0
        }
        if not self.active_population: return {**data, 'inactive': len(self.inactive_population)}

        pop_count = len(self.active_population)
        for p in self.active_population:
            if p.strategy == 'cautious': data['cautious'] += 1
            else: data['readthrough'] += 1
            data['avg_dna_len'] += len(p.dna_sequence)
            for trna_type in p.trna_pool:
                if f'avg_tRNA_{trna_type}' in data:
                    data[f'avg_tRNA_{trna_type}'] += p.trna_pool[trna_type]

        for key in data:
            if key.startswith('avg_'): data[key] /= pop_count
        data['inactive'] = len(self.inactive_population)
        return data

# --- UNCHANGED PLOTTING FUNCTION ---
def plot_simulation_details(results_df, replica_title=""):
    if results_df.empty: print("No data to plot."); return
    fig, axs = plt.subplots(4, 1, figsize=(15, 22), sharex=True, gridspec_kw={'hspace': 0.45})
    fig.suptitle(f'Simulation Detailed Results: {replica_title}', fontsize=16)
    colors = {'cautious': 'orange', 'readthrough': 'purple'}

    axs[0].plot(results_df['time'], results_df['cautious'], label='Cautious', color=colors['cautious'])
    axs[0].plot(results_df['time'], results_df['readthrough'], label='Reckless', color=colors['readthrough'])
    axs[0].set_title('Active Population by Strategy'); axs[0].set_ylabel('Count'); axs[0].legend()

    axs[1].plot(results_df['time'], results_df['avg_dna_len'], color='green')
    axs[1].set_title('Average DNA Length in Population'); axs[1].set_ylabel('Nucleotides')

    for trna_type in ['Trp', 'Tyr', 'Cys', 'Phe', 'Met', 'His', 'Stop']:
        col_name = f'avg_tRNA_{trna_type}'
        if col_name in results_df.columns:
            axs[2].plot(results_df['time'], results_df[col_name], label=f'tRNA-{trna_type}')
    axs[2].set_title('Average tRNA Pool Levels'); axs[2].set_ylabel('Available Units'); axs[2].legend(ncol=4)

    env_data = Environment(np.random.default_rng(), 10)
    uv_plot_len = min(len(results_df), len(env_data.uv_intensity))
    axs[3].plot(results_df['time'][:uv_plot_len], env_data.uv_intensity[:uv_plot_len], color='gray', alpha=0.5, label='UV')
    axs[3].set_title('Environmental UV'); axs[3].set_ylabel('UV Intensity')

    for ax in axs: ax.grid(True, linestyle=':', linewidth='0.5', color='gray')
    plt.tight_layout(rect=[0, 0, 1, 0.96]); plt.show(); plt.close(fig)

# --- NEW & IMPROVED SIMULATION AND ORCHESTRATOR FUNCTIONS ---

def run_single_simulation(steps_per_day, rng, shuffle_rng, initial_dna, checkpoint_path, final_data_path, resume, force_fresh):
    """
    Runs a single simulation replica with save/resume functionality.
    """
    colony, data_log, start_step = None, [], 0
    MAX_STEPS = 50000
    SAVE_INTERVAL = 1000  # Save progress every 1000 steps

    # Logic to handle resuming from a checkpoint
    if resume and not force_fresh and os.path.exists(checkpoint_path):
        try:
            print(f"✅ Resuming replica from checkpoint: {os.path.basename(checkpoint_path)}")
            with open(checkpoint_path, 'rb') as f:
                state = pickle.load(f)
            colony = state['colony']
            data_log = state['data_log']
            start_step = state['last_step'] + 1
            rng.bit_generator.state = state['rng_state']
            shuffle_rng.setstate(state['shuffle_rng_state'])
        except (EOFError, KeyError) as e:
            print(f"⚠️ Checkpoint file corrupted ({e}). Starting fresh.")
            os.remove(checkpoint_path) # Delete corrupted file
            start_step = 0

    if colony is None: # If not resumed, start fresh
        print("🚀 Starting replica fresh.")
        colony = Colony(Environment(rng, steps_per_day), initial_dna, 5000, steps_per_day, rng, shuffle_rng)
        data_log = []
        start_step = 0

    progress_bar = tqdm(desc="Simulating Replica", total=MAX_STEPS, initial=start_step, leave=False)
    step_count = start_step

    for step_count in range(start_step, MAX_STEPS):
        agg_data = colony.get_aggregated_data()
        c_count, r_count = agg_data.get('cautious', 0), agg_data.get('readthrough', 0)

        if step_count > start_step: # Don't log the state we just loaded
             data_log.append({'time': colony.env.time, **agg_data})

        if step_count > 1000 and (c_count == 0 or r_count == 0):
            print("\nOne strategy dominated. Ending run early.")
            break

        colony.step(step_count)
        progress_bar.update(1)
        if step_count % 50 == 0:
            progress_bar.set_postfix_str(f"Cautious: {c_count}, Reckless: {r_count}")

        # Save a checkpoint periodically
        if step_count > 0 and step_count % SAVE_INTERVAL == 0:
            state = {
                'colony': colony, 'data_log': data_log, 'last_step': step_count,
                'rng_state': rng.bit_generator.state, 'shuffle_rng_state': shuffle_rng.getstate()
            }
            with open(checkpoint_path, 'wb') as f:
                pickle.dump(state, f)

    progress_bar.close()
    details_df = pd.DataFrame(data_log)
    details_df.to_pickle(final_data_path) # Save final complete data

    last_data = details_df.iloc[-1] if not details_df.empty else {}
    summary = {
        'Winner': "Cautious" if last_data.get('readthrough', 0) == 0 else "Reckless" if last_data.get('cautious', 0) == 0 else "Tie/Limit",
        'Duration': step_count,
        'Final Cautious': last_data.get('cautious', 0),
        'Final Reckless': last_data.get('readthrough', 0)
    }
    return summary, details_df

def main_orchestrator(num_replicas, steps_per_day, force_fresh_start=False, monitor_plot_only=False):
    """
    Manages the entire experiment, including saving, resuming, and plotting.

    Args:
        num_replicas (int): Number of simulation copies to run.
        steps_per_day (int): Simulation steps that constitute one day.
        force_fresh_start (bool): If True, ignores saved states and starts over.
        monitor_plot_only (bool): If True, skips simulation and only plots final results.
    """
    PRIMORDIAL_DNA = "ATGTGTTTA" + "GGT"
    SAVE_DIR = "/content/drive/My Drive/QuantumSimResults"
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"📂 Results and checkpoints will be saved in: {SAVE_DIR}")

    all_summaries = []

    if monitor_plot_only:
        print("\n---  MONITOR MODE: Plotting results from saved data ---")
        for i in range(1, num_replicas + 1):
            final_data_path = os.path.join(SAVE_DIR, f"replica_{i}_final_data.pkl")
            if os.path.exists(final_data_path):
                print(f"\n📈 Plotting Replica {i}...")
                details_df = pd.read_pickle(final_data_path)
                plot_simulation_details(details_df, replica_title=f"Replica {i} (from save)")
            else:
                print(f"\n❌ No final data found for Replica {i} at {final_data_path}")
        return # Exit after plotting

    # --- Simulation Mode ---
    print(f"\n--- Starting new experiment with {num_replicas} replicas. ---")
    if force_fresh_start:
        print("⚠️ Forced Fresh Start: All previous checkpoints will be ignored.")

    for i in range(1, num_replicas + 1):
        print(f"\n--- Processing Replica {i}/{num_replicas} ---")
        checkpoint_path = os.path.join(SAVE_DIR, f"replica_{i}_checkpoint.pkl")
        final_data_path = os.path.join(SAVE_DIR, f"replica_{i}_final_data.pkl")

        q_rng = QuantumRandomGenerator(29)
        quantum_seed = q_rng.get_seed()
        sim_rng = np.random.default_rng(seed=quantum_seed)
        shuffle_rng = random.Random(quantum_seed)

        # If forcing a fresh start, remove old checkpoints for this replica
        if force_fresh_start and os.path.exists(checkpoint_path):
            os.remove(checkpoint_path)

        summary, details_df = run_single_simulation(
            steps_per_day, sim_rng, shuffle_rng, PRIMORDIAL_DNA,
            checkpoint_path=checkpoint_path,
            final_data_path=final_data_path,
            resume=True,
            force_fresh=force_fresh_start
        )
        summary['Replica'] = i
        all_summaries.append(summary)
        plot_simulation_details(details_df, replica_title=f"Replica {i}")

    print("\n\n" + "="*50 + "\n" + " EXPERIMENT SUMMARY ".center(50, "=") + "\n" + "="*50)
    if all_summaries:
        summary_df = pd.DataFrame(all_summaries).set_index('Replica')
        print(summary_df.to_string())
        # Save summary to file
        summary_df.to_csv(os.path.join(SAVE_DIR, "experiment_summary.csv"))
        print(f"\n📄 Summary saved to {os.path.join(SAVE_DIR, 'experiment_summary.csv')}")
    print("="*50)


# --- HOW TO RUN ---
if __name__ == "__main__":
    # Option 1: Run the simulation normally (will resume if checkpoints exist)
    main_orchestrator(num_replicas=3, steps_per_day=10)

    # Option 2: Force a fresh start, deleting any progress
    # main_orchestrator(num_replicas=3, steps_per_day=10, force_fresh_start=True)

    # Option 3: Don't simulate, just plot the results from the last completed run
    # main_orchestrator(num_replicas=3, steps_per_day=10, monitor_plot_only=True)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 Results and checkpoints will be saved in: /content/drive/My Drive/QuantumSimResults

--- Starting new experiment with 3 replicas. ---

--- Processing Replica 1/3 ---
🚀 Starting replica fresh.


Simulating Replica:   0%|          | 0/50000 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1-435689509.py", line 394, in <cell line: 0>
    main_orchestrator(num_replicas=3, steps_per_day=10)
  File "/tmp/ipython-input-1-435689509.py", line 370, in main_orchestrator
    summary, details_df = run_single_simulation(
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1-435689509.py", line 285, in run_single_simulation
    agg_data = colony.get_aggregated_data()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-1-435689509.py", line None, in get_aggregated_data
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_t